<a href="https://colab.research.google.com/github/andre-arantes/ia/blob/master/tp1/etapa1/Tratamento_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Previsão de Diabetes

Este projeto tem como objetivo prever a probabilidade de uma pessoa ter diabetes com base em vários fatores, como idade, nível de HbA1c, nível de glicose no sangue e gênero. O conjunto de dados usado neste projeto é o Diabetes Prediction Dataset.

Usaremos o algoritmo Random Forest Classifier para treinar nosso modelo e prever a probabilidade de diabetes. Também usaremos várias técnicas de pré-processamento de dados, como padronização e subamostragem, para melhorar a precisão do nosso modelo.

In [ ]:
# Importando bibliotecas
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
# Carregando os dados
base = pd.read_csv('/content/sample_data/diabetes_prediction_dataset.csv')

# Ler dados da base

In [ ]:
base.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
# checar existencias de campos nulos
base.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [ ]:
# checar natureza dos dados
for i in base.columns:
    print(base[i].value_counts())
    print('\n')

Female    16178
Male      11345
Other         5
Name: gender, dtype: int64


80.00    1541
51.00     480
48.00     452
47.00     438
55.00     431
         ... 
1.48       21
1.88       20
0.16       16
0.40       14
0.08        6
Name: age, Length: 102, dtype: int64


0    25403
1     2125
Name: hypertension, dtype: int64


0    26460
1     1068
Name: heart_disease, dtype: int64


never          9799
No Info        9723
former         2635
current        2540
not current    1746
ever           1085
Name: smoking_history, dtype: int64


27.32    7031
27.80      35
25.00      35
25.15      34
24.36      31
         ... 
63.74       1
55.26       1
34.71       1
40.30       1
41.35       1
Name: bmi, Length: 3418, dtype: int64


5.7    2338
5.8    2299
6.0    2286
6.1    2283
6.5    2273
6.2    2271
6.6    2255
3.5    2176
4.0    2111
4.8    2073
4.5    2036
5.0    2020
8.2     198
7.5     191
9.0     190
8.8     186
7.0     172
6.8     170
Name: HbA1c_level, dtype: int64


160    2185
1

#Pré-processamento dos dados

A classe `StandardScaler` realiza a padronização dos dados, transformando as features para que tenham média zero e desvio padrão igual a um. Isso ajuda a evitar que features com valores muito grandes ou muito pequenos dominem o modelo de aprendizado de máquina.

In [ ]:
from sklearn.preprocessing import StandardScaler
# remover campo 'smoking_history' devido a quantidade de informações faltantes
base.drop('smoking_history',axis=1,inplace=True)

# transformar genero em valor numerico
base['gender']=base['gender'].map({'Female':0,'Male':1,'Other':2})

sc=StandardScaler()

# Padronização das features 'age', 'HbA1c_level' e 'blood_glucose_level' usando a classe StandardScaler
d = sc.fit_transform(base[['age', 'HbA1c_level', 'blood_glucose_level']])

# Remoção das features originais do conjunto de dados
base.drop(['age', 'HbA1c_level', 'blood_glucose_level'], axis=1, inplace=True)

# Criação de um novo DataFrame com as features padronizadas
d2 = pd.DataFrame(d, columns=['age', 'HbA1c_level', 'blood_glucose_level'])

# Concatenação do novo DataFrame com o conjunto de dados original
final=pd.concat([d2,base],axis=1)

NameError: ignored

#Divisão dos dados em treino e teste

In [ ]:
# criar testes e treinos
x=final.drop('diabetes',axis=1)
y=final['diabetes']
X_treino,X_teste,y_treino,y_teste=train_test_split(x,y,test_size=0.2)

#Treinamento do modelo

In [ ]:
# Equilibra as classes no treinamento, preservando 40% das amostras da classe comum em problemas com classes desequilibradas.
us=RandomUnderSampler(sampling_strategy=0.4)
X_treino,y_treino=us.fit_resample(X_treino,y_treino)

#Avaliação do modelo

In [ ]:
# rodar random Forest
modelo = RandomForestClassifier()
modelo.fit(X_treino,y_treino)
p = modelo.predict(X_teste)
print(classification_report(y_teste,p))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      5062
           1       0.66      0.81      0.73       444

    accuracy                           0.95      5506
   macro avg       0.82      0.89      0.85      5506
weighted avg       0.96      0.95      0.95      5506

